In [1]:
import rasterio.features
import rasterio.plot as plot
import json
import os
import sys
import numpy as np

import WINMOL_Analyzer as Analyzer
from WINMOL_Analyzer import Timer

from Config import Config

# Root directory of the project
ROOT_DIR = os.path.abspath(".//")

sys.path.append(ROOT_DIR)

# Directory to save logs and trained model
model_dir = os.path.join(ROOT_DIR, "model//")

# Directory to save logs and trained model
pred_dir = os.path.join(ROOT_DIR, "pred//")

# Directory to save logs and trained model
output_dir = os.path.join(ROOT_DIR, "output//")

In [2]:
config = Config()
config.display()


Configurations:
IMG_bit                        8
IMG_height                     512
IMG_width                      512
max_distance                   8
max_tree_height                32
min_length                     2.0
n_Channels                     3
num_classes                    1
overlapp_pred                  8
tile_size                      15
tolerance_angle                7




In [3]:
img_list=os.listdir(pred_dir)
img_list= [i for i in img_list if i.endswith((".tiff",".tif")) ]
img_name=img_list[0]
img_name

'pred_20220209_Bremerhagen_2.tiff'

In [5]:
tt = Timer()
tt.start()

pred, profile =Analyzer.IO.load_stem_map(pred_dir+img_name)
segments = Analyzer.Skel.find_segments(pred, config, profile)
segments = Analyzer.Vec.restore_geoinformation(segments, config, profile)
stems = Analyzer.Vec.build_stem_parts(segments)
stems = Analyzer.Vec.connect_stems(stems,config)  
endnodes = Analyzer.Vec.rebuild_endnodes_from_stems(stems)   
stems = Analyzer.Quant.quantify_stems(stems, pred, profile)
Analyzer.IO.stems_to_geojson(stems, output_dir+ os.path.splitext(os.path.basename(img_name))[0])

tt.stop()    

#######################################################
#######################################################

P:\WINMOL_Analyzer\pred//pred_20220209_Bremerhagen_2.tiff

#######################################################
Skeletonize Image
Elapsed time: 0.5253 seconds
#######################################################

#######################################################
Splitting the skeleton into segments and detecting endnodes
Dense nodes removed:  12
Brachpoints removed:  261
Detected endnodes:  454
Elapsed time: 8.2224 seconds
#######################################################

#######################################################
Find connected segments in the skeleton
Initial length of skeleton:  9921
Number of endnodes 454
Minimum length in pixel:  17
Detected skeleton segments:  93
Elapsed time: 17.5046 seconds
#######################################################

#######################################################
#Refining and sorting out skeleton